In [2]:
import numpy as np
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import math

# Feature Engineering

In [46]:
df = pd.read_csv('monthly_2014_8_2020_7_4k_drugs.csv')
def fun(row):
    return str(row['Year'])+'-' + str(row['Month'])
df['Date'] = df.apply(lambda x:fun(x),axis = 1)
df.Date = pd.to_datetime(df.Date)
df.sort_values(['Date'],inplace = True)

In [47]:
"""
df = df[df['Product Launch Date']!='Unspecified']
df['Product Launch Date'] = pd.to_datetime(df['Product Launch Date'])
df['Launch_Year'] = df['Product Launch Date'].apply(lambda x:x.year)
df['Time_from_launch'] = df.Year - df['Launch_Year']
"""

In [5]:
"""
df = df[df['Estimated LOE Date']!='Unspecified']
df['Maturity_Year'] = df['Estimated LOE Date'].apply(lambda x:int('20'+x[4:]))
df['Time_to_maturity'] = df['Maturity_Year']- df.Year
"""

"\ndf = df[df['Estimated LOE Date']!='Unspecified']\ndf['Maturity_Year'] = df['Estimated LOE Date'].apply(lambda x:int('20'+x[4:]))\ndf['Time_to_maturity'] = df['Maturity_Year']- df.Year\n"

# Filter
1. Drugs have WAC data from 2014.8 to 2020.7
2. Drugs have TRx data from 2014.8 to 2020.7
3. Drop duplicates (one NDC corresponding to two manufacturers)

In [48]:
## Filter two
df.dropna(subset=['TRx'],inplace = True)
## Filter one
list1 = df[(df.Year == 2020) & (df.Month == 7)].NDC.unique()
list2 = df[(df.Year == 2014) & (df.Month == 8)].NDC.unique()
selected_NDC = list(set(list1).intersection(list2)) 
df = df[df.NDC.isin(selected_NDC)]

In [49]:
## Filter top 10 classes
temp = df.groupby('Major Class')['NDC'].nunique().reset_index().sort_values('NDC',ascending =False)
list3 = temp['Major Class'].tolist()[:10]
df = df[df['Major Class'].isin(list3)]

In [50]:
## Drop duplicates
df = df[~df.duplicated()]
df.drop_duplicates(subset=df.columns.difference(['Manufacturer']),inplace = True)

# Modeling

In [42]:
def predict_pct_month(df,full):
        df['TRx']= df['TRx'].rolling(6).mean().shift()## Take previous 6 months average 
        df.TRx = df.TRx.apply(lambda x:math.log(x)) ## Take logarithm
        df.dropna(inplace = True)
        date = max(df.ds)
        prophet = Prophet(changepoint_prior_scale=0.05, daily_seasonality=False,weekly_seasonality=False,yearly_seasonality=True)
        prophet.add_regressor('TRx')
        prophet.fit(df)
        build_forecast = prophet.make_future_dataframe(periods=12, freq='M')
        full['TRx']= full['TRx'].rolling(6).mean().shift()
        full.TRx = full.TRx.apply(lambda x:math.log(x))
        full.dropna(inplace = True)
        build_forecast['TRx'] = full['TRx'].tolist()
        forecast = prophet.predict(build_forecast)
        forecast = forecast[forecast.ds>=date]
        forecast.reset_index(inplace = True)
        forecast['pct_change'] = forecast.yhat.apply(lambda x:(x-df.iloc[-1].y)/df.iloc[-1].y)
        ##month_ = forecast.iloc[forecast['pct_change'].idxmax()].ds.month
        previous_year = df[df.ds>= '2018-08-01']
        previous_year.reset_index(inplace = True)
        previous_year['pct_change'] = previous_year.y.pct_change()
        ## Use this year's change point as prediction for next year
        ## If this year the drug price does not increase, go back to previous year
        if previous_year['pct_change'].mean() == 0:
            previous_year = df[df.ds>= '2017-08-01']
            previous_year.reset_index(inplace = True)
            previous_year['pct_change'] = previous_year.y.pct_change()
            month_ = previous_year.iloc[previous_year['pct_change'].idxmax()].ds.month
        else:
            month_ = previous_year.iloc[previous_year['pct_change'].idxmax()].ds.month
        pct_ = forecast[forecast['pct_change']>0.01]['pct_change'].mean() ## Take average pct_change to filter out fluctuations
        if math.isnan(pct_):
            pct_ = 0
        return month_,pct_
   

In [11]:
def actual_pct_month(df):
    df.reset_index(inplace = True)
    df['pct_change'] = df.y.pct_change()
    month_true = df.iloc[df['pct_change'].idxmax()].ds.month
    pct_true = df['pct_change'].max()
    return month_true,pct_true
    

In [12]:
##Get evaluation
def evaluation_result(dict_):
    diff_month = []
    diff_pct = []
    for key in dict_.keys():
        if dict_[key][3]!=0: ## Rule out drug that 
            temp1 = abs(dict_[key][0]-dict_[key][1])
            temp2 = min(dict_[key][0],dict_[key][1])+12-max(dict_[key][0],dict_[key][1])
            temp = min(temp1,temp2) 
            diff_month.append(temp)
            diff_pct.append(abs(dict_[key][2]-dict_[key][3]))
    
    diff_month = sum(diff_month)/len(diff_month)
    diff_pct = sum(diff_pct)/len(diff_pct)
    return diff_month,diff_pct
        

In [13]:
## Most LIKELY MONTH
"""
def Mode_month(dict_):
    list_ = []
    for key in dict_.keys():
        if dict_[key][1]>0.01:
            list_.append(dict_[key][0])
    data = Counter(list_)  # Returns all unique items and their counts
    mode = data.most_common(1)[0][0]
    mode_percentage = data.most_common(1)[0][1]/len(list_)
    return mode,mode_percentage
"""

In [43]:
def pipe(class_name):
    class_ = df[df['Major Class']==class_name]
    dict_ = {}
    for NDC in class_.NDC.unique():
        try:
            temp = class_[class_.NDC == NDC][['Date','WAC','TRx']]
            temp.columns = ['ds','y','TRx']
            train = temp[temp.ds< '2019-08-01']
            test = temp[temp.ds>= '2019-08-01']
            c,d = actual_pct_month(test)
            if d == 0:
                pass
            a,b = predict_pct_month(train,temp)
            dict_[NDC] = [a,c,b,d]
        except:
            print(NDC)
    return evaluation_result(dict_)


In [51]:
res = {}
for major_class in df['Major Class'].unique():
    res[major_class] = pipe(major_class)

409161050
409125301
409120301
87607005
9501401
64764012103
597019190
78040805
69600125
6391568
777310507
50458030050
10337035860
85092401
187518006
99207001010
51672526303
43538091012
43538091112
43538092060
69260066
69265072
61570011001
49275066
25171002


INFO:fbprophet:n_changepoints greater than number of observations. Using 19.


25191131


INFO:fbprophet:n_changepoints greater than number of observations. Using 19.


140000614
206886001
206886201


INFO:fbprophet:n_changepoints greater than number of observations. Using 19.
INFO:fbprophet:n_changepoints greater than number of observations. Using 23.


67308010106
78056661
78056761
50242007001
13013202


In [52]:
res ## First is average month_diff across drug class 
## Second is average pct_change diff across drug class

{'PAIN': (0.82, 0.021058035381611703),
 'ANTIDIABETICS': (0.0, 0.014584455460854481),
 'NERVOUS SYSTEM DISORDERS': (0.32142857142857145, 0.07864279025801002),
 'MENTAL HEALTH': (0.3614457831325301, 0.015295869975106078),
 'DERMATOLOGICS': (0.4166666666666667, 0.011194654384396043),
 'ANTIHYPERTENSIVES, PLAIN & COMBO': (0.4782608695652174,
  0.016283275044742362),
 'RESPIRATORY AGENTS': (0.11538461538461539, 0.009669032572607412),
 'OTHER CNS': (0.6216216216216216, 0.01608438452780767),
 'ANTIBACTERIALS': (1.6666666666666667, 0.04080531965866624),
 'ONCOLOGICS': (0.7738095238095238, 0.013926514112247126)}

In [29]:
res ## Previous version results without optimization

{'PAIN': (2.42, 0.025552123494570964),
 'ANTIDIABETICS': (1.5892857142857142, 0.0168335097490989),
 'NERVOUS SYSTEM DISORDERS': (2.232142857142857, 0.017448531989300495),
 'MENTAL HEALTH': (1.7590361445783131, 0.022394225563417733),
 'DERMATOLOGICS': (2.25, 0.02312078100469477),
 'ANTIHYPERTENSIVES, PLAIN & COMBO': (2.717391304347826, 0.03780697051068805),
 'RESPIRATORY AGENTS': (2.1923076923076925, 0.01315855073442027),
 'OTHER CNS': (2.5675675675675675, 0.03197545114105418),
 'ANTIBACTERIALS': (2.2222222222222223, 0.06700542161188248),
 'ONCOLOGICS': (2.7261904761904763, 0.021971020122865566)}